In [42]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.test.utils import common_corpus, common_dictionary
from gensim.corpora.dictionary import Dictionary
import nltk
from nltk.corpus import stopwords
import json
import re

def clean_corpus(corpus):
    domain_stop_words = {'someone', 'work', 'things', 'like', 'would', 'going', 'feel', 'still', 'little', 'test', 'want', 'since', 'really', 'also', 'even', 'could', 'getting', 'make', 'though', 'people'}
    cleaned = []
    for document in corpus:
        words = nltk.word_tokenize(document) #tokenize the document content
        words = [w.lower() for w in words if w.lower() not in domain_stop_words and w.lower() not in stopwords.words('english') and w.isalpha() and len(w) > 3]
        cleaned.append(words)
    vocab = []
    for text in cleaned:
      for word in text:
        vocab.append(word)
    return cleaned,vocab


# filepath = '/Users/angela/Documents/NLPResults/Datasets/BT_only_new.json'
# filepath = '/Users/angela/Documents/NLPResults/Datasets/chatgpt_only copy.json'
# filepath = '/Users/angela/Documents/NLPResults/Datasets/scraped_only.json'
filepath = 'data_withposts/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS_additional_emotions.json'
with open(filepath) as f:
    dataset = json.load(f)

emotionList = ["trust", "disgust", "joy", "confusion", "resilience"]
for emotion in emotionList:
    documents = []
    print("emotion: ", emotion)
    for k in dataset:
        summary = None
        emo = 0
        for annotation in dataset[k]['Annotations']:
            for hit in dataset[k]['Annotations'][annotation]:
                if hit['Emotion'] == emotion:
                    emo = 1
                    summary = hit['Abstractive']
        if emo == 1:
            # print(summary)
            # documents.append(dataset[k]['Reddit Post'][0]) 
            documents.append(summary) 

    clean_corp,vocab = clean_corpus(documents)
    unique_vocab_dict = Dictionary(clean_corp)
    topic_corpus = [unique_vocab_dict.doc2bow(doc) for doc in clean_corp]
    lda_model = LdaModel(topic_corpus,id2word=unique_vocab_dict,num_topics=1,passes=10) #build the lda model

    topics = lda_model.print_topics(num_topics=1, num_words=20)
    for topic in topics:
        words = [word.split('"')[1] for word in topic[1].split() if '"' in word]
        print(words)



emotion:  trust
['trust', 'covid', 'help', 'vaccine', 'know', 'information', 'vaccines', 'accept', 'pandemic', 'able', 'others', 'vaccinated', 'better', 'virus', 'good', 'risk', 'catching', 'need', 'opinions', 'knowing']
emotion:  disgust
['disgusted', 'covid', 'pandemic', 'vaccine', 'vaccinated', 'mask', 'disgusting', 'vaccines', 'enough', 'time', 'vaccination', 'disgusts', 'care', 'need', 'taking', 'around', 'virus', 'family', 'tested', 'health']
emotion:  joy
['happy', 'covid', 'vaccinated', 'pandemic', 'vaccine', 'good', 'positive', 'time', 'news', 'able', 'life', 'back', 'better', 'virus', 'normal', 'everyone', 'vaccines', 'feeling', 'shot', 'much']
emotion:  confusion
['confused', 'whether', 'covid', 'symptoms', 'vaccine', 'safe', 'positive', 'booster', 'risk', 'vaccinated', 'negative', 'tested', 'omicron', 'family', 'need', 'tests', 'exposed', 'stay', 'long', 'quarantine']
emotion:  resilience
['pandemic', 'hope', 'good', 'positive', 'news', 'year', 'vaccines', 'covid', 'optimis